In [1]:
import os
print(os.listdir())  # Lists all files in the current directory


['img_329.jpg', 'img_473.jpg', 'img_315.jpg', 'img_301.jpg', 'img_467.jpg', 'img_10.jpg', 'img_1450.jpg', 'img_1322.jpg', 'img_38.jpg', 'img_881.jpg', 'img_117.jpg', 'img_103.jpg', 'img_1108.jpg', 'img_856.jpg', 'img_842.jpg', 'img_1134.jpg', 'img_1120.jpg', 'img_739.jpg', 'img_705.jpg', 'img_1097.jpg', 'img_936.jpg', 'img_1068.jpg', 'img_1040.jpg', 'img_1054.jpg', 'Cisco Packet Tracer 8.0.0', 'img_249.jpg', 'img_507.jpg', 'img_1281.jpg', 'img_1.jpg', 'img_261.jpg', 'Solution.java', 'img_1243.jpg', 'EightPuzzleGreedySearch.java', 'img_1294.jpg', 'img_512.jpg', 'img_506.jpg', 'img_260.jpg', 'img_1280.jpg', 'img_248.jpg', 'Music', 'img_1041.jpg', 'img_937.jpg', 'img_923.jpg', 'img_710.jpg', 'img_1082.jpg', 'img_738.jpg', 'img_1121.jpg', 'img_1135.jpg', 'img_1109.jpg', 'GreedyBestFirstSearch.java', 'img_670.jpg', 'img_880.jpg', 'img_658.jpg', 'img_894.jpg', 'img_1445.jpg', 'img_39.jpg', 'img_1451.jpg', 'img_11.jpg', 'img_300.jpg', 'img_466.jpg', 'img_314.jpg', 'img_328.jpg', 'img_458.jpg'

In [2]:
import zipfile

zip_path = "StateFarmDataset.zip"  # Use the new name
extract_path = "StateFarmDataset"  # Destination folder

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("Extraction Successful!")


Extraction Successful!


In [3]:
!pip install opencv-python


Defaulting to user installation because normal site-packages is not writeable


In [4]:
import os
import shutil

# Paths
original_path = "StateFarmDataset/State Farm Dataset/imgs/train"
filtered_path = "StateFarmDataset/filtered_drinking"

# Define new class mapping
class_mapping = {
    "Safe": ["c0"],
    "Drinking": ["c6"]
}

# Create destination folders and copy files
for new_class, original_classes in class_mapping.items():
    dest_dir = os.path.join(filtered_path, new_class)
    os.makedirs(dest_dir, exist_ok=True)

    for orig_class in original_classes:
        src_dir = os.path.join(original_path, orig_class)
        if not os.path.exists(src_dir):
            print(f"⚠️ Source folder missing: {src_dir}")
            continue

        files = os.listdir(src_dir)
        for file in files:
            src_file = os.path.join(src_dir, file)
            dest_file = os.path.join(dest_dir, file)

            try:
                shutil.copy2(src_file, dest_file)
            except Exception as e:
                print(f"❌ Error copying {file}: {e}")

print("✅ Drinking detection dataset created at:", filtered_path)


✅ Drinking detection dataset created at: StateFarmDataset/filtered_drinking


In [5]:
 import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import os

# Paths
filtered_path = "StateFarmDataset/filtered_drinking"

# Image Data Generator
img_size = (224, 224)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=10,
    zoom_range=0.1,
    horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
    filtered_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    filtered_path,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

# Define model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')  # 2 classes: Safe, Drinking
])

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

# Save model
model.save("drinking_detection_model.h5")

print("✅ Model trained and saved as drinking_detection_model.h5")


/Users/shubhrabajpai/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


Found 3852 images belonging to 2 classes.
Found 962 images belonging to 2 classes.


/Users/shubhrabajpai/Library/Python/3.9/lib/python/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/Users/shubhrabajpai/Library/Python/3.9/lib/python/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 141s 1s/step - accuracy: 0.6168 - loss: 0.7365 - val_accuracy: 0.9667 - val_loss: 0.1259
Epoch 2/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 139s 1s/step - accuracy: 0.9548 - loss: 0.1344 - val_accuracy: 0.9792 - val_loss: 0.0674
Epoch 3/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 139s 1s/step - accuracy: 0.9774 - loss: 0.0685 - val_accuracy: 0.9792 - val_loss: 0.0663
Epoch 4/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 142s 1s/step - accuracy: 0.9848 - loss: 0.0435 - val_accuracy: 0.9854 - val_loss: 0.0412
Epoch 5/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 143s 1s/step - accuracy: 0.9894 - loss: 0.0398 - val_accuracy: 0.9917 - val_loss: 0.0250
Epoch 6/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 146s 1s/step - accuracy: 0.9913 - loss: 0.0367 - val_accuracy: 0.9938 - val_loss: 0.0204
Epoch 7/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 142s 1s/step - accuracy: 0.9961 - loss: 0.0197 - val_accuracy: 0.9834 - val_loss: 0.0506
Epoch 8/10
121/121 ━━━━━━━━━━━━━━━━━━━━ 153s 1s/step - accuracy: 0.9910 - loss: 0.0310 - val_accu

✅ Model trained and saved as drinking_detection_model.h5


In [6]:
import cv2
import tensorflow as tf
import numpy as np

# Load trained model
model = tf.keras.models.load_model('drinking_detection_model.h5')

# Define class labels (order must match the training generator)
class_labels = ['Drinking', 'Safe']  # Check this matches what flow_from_directory returned

# Image size (should match training input)
img_size = (224, 224)

def test_drinking_with_video(video_source=0):
    cap = cv2.VideoCapture(video_source)

    if not cap.isOpened():
        print("❌ Error: Could not open video stream.")
        return

    while True:
        ret, frame = cap.read()
        if not ret:
            print("🚫 End of video or can't read frame.")
            break

        # Preprocess frame
        resized_frame = cv2.resize(frame, img_size)
        normalized_frame = resized_frame / 255.0
        input_tensor = np.expand_dims(normalized_frame, axis=0)

        # Predict
        prediction = model.predict(input_tensor)[0]
        predicted_class = np.argmax(prediction)
        confidence = prediction[predicted_class]

        label = class_labels[predicted_class]

        if label == "Drinking":
            display_text = f"{label} ({confidence:.2f})"
            cv2.putText(frame, display_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX,
                        1, (0, 0, 255), 2, cv2.LINE_AA)

        # Show frame
        cv2.imshow("Drinking Detection", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("👋 Exiting.")
            break

    cap.release()
    cv2.destroyAllWindows()

# Run the function
# For webcam: test_drinking_with_video(0)
# For video file: test_drinking_with_video("input_video.mp4")
test_drinking_with_video("input_video.mp4")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━